In [20]:
#import input_net
from os import path
import numpy as np
import pandas as pd
import torch
from pytorch_pretrained_bert import BertModel, BertForSequenceClassification

In [12]:
from os import path
import pandas as pd

In [36]:
TRAIN = "data/train.csv"
INPUT_NET = 'data/input.csv'

if not path.exists(INPUT_NET):
    data = input_net.create_input(TRAIN, INPUT_NET)
else:
    data = pd.read_csv(INPUT_NET)

In [38]:
input_ids = np.array(data.iloc[:,:70])
token_type_ids = np.array(data.iloc[:,70:140])
attention_masks = np.array(data.iloc[:,140:])

tokens_tensor = torch.tensor([input_ids])
segments_tensors = torch.tensor([token_type_ids])
attention_tensors = torch.tensor([attention_masks])

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        

In [43]:
Input = InputFeatures(input_ids=input_ids,input_mask=attention_masks,segment_ids=token_type_ids,label_id=targets)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

In [ ]:
# Mirar com funciona!!
Predictions = model(tokens_tensor,segments_tensors,attention_tensors,targets)